<a href="https://colab.research.google.com/github/2348754114579/Image-and-video-enhancer/blob/main/%F0%9F%87%A7%F0%9F%87%A9%E2%AD%90Image_correction_Gradio_v1%E2%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio --upgrade
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
import numpy as np
import cv2
from io import BytesIO
from scipy.ndimage import gaussian_filter
import gradio as gr

def apply_rgb(image, red_factor, green_factor, blue_factor):
    if image is None:
        return None
    img_array = np.array(image).astype(np.float32)
    img_array[:, :, 0] *= red_factor / 100.0
    img_array[:, :, 1] *= green_factor / 100.0
    img_array[:, :, 2] *= blue_factor / 100.0
    img_array = np.clip(img_array, 0, 255).astype(np.uint8)
    return Image.fromarray(img_array)

def apply_gamma(image, gamma):
    if image is None:
        return None
    img_array = np.array(image).astype(np.float32) / 255.0
    corrected_array = np.power(img_array, 1.0 / gamma)
    corrected_array = np.clip(corrected_array * 255, 0, 255).astype(np.uint8)
    return Image.fromarray(corrected_array)

def apply_contrast(image, contrast):
    if image is None:
        return None
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(contrast)

def apply_brightness(image, brightness):
    if image is None:
        return None
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness)

def apply_hue(image, hue):
    if image is None:
        return None
    img_hsv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2HSV).astype(np.float32)
    h, s, v = cv2.split(img_hsv)
    h += hue
    h = np.clip(h, 0, 180)
    merged_hsv = cv2.merge([h, s, v]).astype(np.uint8)
    return Image.fromarray(cv2.cvtColor(merged_hsv, cv2.COLOR_HSV2RGB))

def apply_vibrance(image, vibrance):
    if image is None:
        return None
    factor = 1 + vibrance

    def vibrance_map(pixel):
        r, g, b = pixel
        gray = 0.299 * r + 0.587 * g + 0.114 * b
        saturation = max(r, g, b) - min(r, g, b)
        scale = factor * (1 - saturation / 255.0) if factor > 0 else factor
        new_r = np.clip(r + (r - gray) * scale, 0, 255).astype(np.uint8)
        new_g = np.clip(g + (g - gray) * scale, 0, 255).astype(np.uint8)
        new_b = np.clip(b + (b - gray) * scale, 0, 255).astype(np.uint8)
        return (new_r, new_g, new_b)

    img = image.copy().convert("RGB")
    pixels = img.load()
    width, height = img.size
    for x in range(width):
        for y in range(height):
            pixels[x, y] = vibrance_map(img.getpixel((x, y)))
    return img

def apply_temperature(image, temperature):
    if image is None:
        return None
    temp = temperature
    img_array = np.array(image).astype(np.float32)
    if temp > 0: # Warmer
        img_array[:, :, 0] = np.clip(img_array[:, :, 0] + temp * 1.5, 0, 255) # Increase Red
        img_array[:, :, 1] = np.clip(img_array[:, :, 1] + temp * 0.5, 0, 255) # Increase Green slightly
    elif temp < 0: # Cooler
        img_array[:, :, 1] = np.clip(img_array[:, :, 1] + temp * 0.5, 0, 255) # Decrease Green slightly
        img_array[:, :, 2] = np.clip(img_array[:, :, 2] - temp * 1.5, 0, 255) # Increase Blue
    return Image.fromarray(img_array.astype(np.uint8))

def apply_blur(image, radius):
    if image is None:
        return None
    return image.filter(ImageFilter.BLUR)

def apply_gaussian_blur(image, radius):
    if image is None:
        return None
    return image.filter(ImageFilter.GaussianBlur(radius=radius))

def apply_sharpen(image, factor):
    if image is None:
        return None
    enhancer = ImageEnhance.Sharpness(image)
    return enhancer.enhance(factor)

def apply_edge_enhance(image):
    if image is None:
        return None
    return image.filter(ImageFilter.EDGE_ENHANCE)

def apply_contour(image):
    if image is None:
        return None
    return image.filter(ImageFilter.CONTOUR)

def apply_detail(image):
    if image is None:
        return None
    return image.filter(ImageFilter.DETAIL)

def apply_smooth(image):
    if image is None:
        return None
    return image.filter(ImageFilter.SMOOTH)

def apply_more_smooth(image):
    if image is None:
        return None
    return image.filter(ImageFilter.SMOOTH_MORE)

def apply_emboss(image):
    if image is None:
        return None
    return image.filter(ImageFilter.EMBOSS)

def apply_find_edges(image):
    if image is None:
        return None
    return image.filter(ImageFilter.FIND_EDGES)

def apply_depth_of_field(image, center_x, center_y, blur_amount, falloff_radius):
    if image is None:
        return None
    img_array = np.array(image).astype(np.float32)
    height, width, _ = img_array.shape
    blurred_image = gaussian_filter(img_array, sigma=[blur_amount, blur_amount, 0])
    output_array = img_array.copy()

    for y in range(height):
        for x in range(width):
            distance = np.sqrt((x - center_x)**2 + (y - center_y)**2)
            if distance > falloff_radius:
                output_array[y, x] = blurred_image[y, x]
            else:
                blur_factor = 1 - (distance / falloff_radius)
                output_array[y, x] = (1 - blur_factor) * img_array[y, x] + blur_factor * blurred_image[y, x]

    return Image.fromarray(np.clip(output_array, 0, 255).astype(np.uint8))

def edit_image(image, red, green, blue, gamma, contrast, brightness, hue, vibrance, temperature, filter_type, blur_radius, gaussian_blur_radius, sharpen_factor, dof_center_x, dof_center_y, dof_blur_amount, dof_falloff_radius):
    edited_image = image
    if edited_image is not None:
        edited_image = apply_rgb(edited_image, red, green, blue)
        edited_image = apply_gamma(edited_image, gamma)
        edited_image = apply_contrast(edited_image, contrast)
        edited_image = apply_brightness(edited_image, brightness)
        edited_image = apply_hue(edited_image, hue)
        edited_image = apply_vibrance(edited_image, vibrance)
        edited_image = apply_temperature(edited_image, temperature)

        if filter_type == "Blur":
            edited_image = apply_blur(edited_image, blur_radius)
        elif filter_type == "Gaussian Blur":
            edited_image = apply_gaussian_blur(edited_image, gaussian_blur_radius)
        elif filter_type == "Sharpen":
            edited_image = apply_sharpen(edited_image, sharpen_factor)
        elif filter_type == "Edge Enhance":
            edited_image = apply_edge_enhance(edited_image)
        elif filter_type == "Contour":
            edited_image = apply_contour(edited_image)
        elif filter_type == "Detail":
            edited_image = apply_detail(edited_image)
        elif filter_type == "Smooth":
            edited_image = apply_smooth(edited_image)
        elif filter_type == "More Smooth":
            edited_image = apply_more_smooth(edited_image)
        elif filter_type == "Emboss":
            edited_image = apply_emboss(edited_image)
        elif filter_type == "Find Edges":
            edited_image = apply_find_edges(edited_image)
        elif filter_type == "Depth of Field":
            if dof_center_x is not None and dof_center_y is not None and dof_blur_amount is not None and dof_falloff_radius is not None:
                edited_image = apply_depth_of_field(edited_image, dof_center_x, dof_center_y, dof_blur_amount, dof_falloff_radius)

    return edited_image

with gr.Blocks() as iface:
    image_input = gr.Image(label="Upload Image", type="pil")
    with gr.Row():
        with gr.Column():
            rgb_red = gr.Slider(0, 200, step=1, value=100, label="Red")
            rgb_green = gr.Slider(0, 200, step=1, value=100, label="Green")
            rgb_blue = gr.Slider(0, 200, step=1, value=100, label="Blue")
            gamma_slider = gr.Slider(0.1, 5.0, step=0.1, value=1.0, label="Gamma")
            contrast_slider = gr.Slider(0.0, 3.0, step=0.1, value=1.0, label="Contrast")
            brightness_slider = gr.Slider(0.0, 3.0, step=0.1, value=1.0, label="Brightness")
            hue_slider = gr.Slider(-180, 180, step=1, value=0, label="Hue")
            vibrance_slider = gr.Slider(-2.0, 2.0, step=0.1, value=0.0, label="Vibrance")
            temperature_slider = gr.Slider(-100, 100, step=5, value=0, label="Temperature")

            filter_choices = ["None", "Blur", "Gaussian Blur", "Sharpen", "Edge Enhance", "Contour", "Detail", "Smooth", "More Smooth", "Emboss", "Find Edges", "Depth of Field"]
            filter_dropdown = gr.Dropdown(choices=filter_choices, value="None", label="Filter")

            blur_radius_slider = gr.Slider(1, 10, step=1, value=2, label="Blur Radius", visible=False)
            gaussian_blur_radius_slider = gr.Slider(1, 10, step=1, value=2, label="Gaussian Blur Radius", visible=False)
            sharpen_factor_slider = gr.Slider(0.0, 5.0, step=0.1, value=1.0, label="Sharpen Factor", visible=False)

            with gr.Group(visible=False) as dof_group:
                dof_center_x_slider = gr.Slider(0.0, 1.0, step=0.01, value=0.5, label="DoF Center X (Normalized)")
                dof_center_y_slider = gr.Slider(0.0, 1.0, step=0.01, value=0.5, label="DoF Center Y (Normalized)")
                dof_blur_amount_slider = gr.Slider(0.0, 10.0, step=0.1, value=2.0, label="DoF Blur Amount")
                dof_falloff_radius_slider = gr.Slider(0.0, 1.0, step=0.01, value=0.2, label="DoF Falloff Radius (Normalized)")

        image_output = gr.Image(label="Edited Image", type="pil")

    def update_filter_visibility(filter_type):
        blur_visible = filter_type == "Blur"
        gaussian_blur_visible = filter_type == "Gaussian Blur"
        sharpen_visible = filter_type == "Sharpen"
        dof_visible = filter_type == "Depth of Field"
        return {
            blur_radius_slider: gr.update(visible=blur_visible),
            gaussian_blur_radius_slider: gr.update(visible=gaussian_blur_visible),
            sharpen_factor_slider: gr.update(visible=sharpen_visible),
            dof_group: gr.update(visible=dof_visible),
        }

    def process_image(image, red, green, blue, gamma, contrast, brightness, hue, vibrance, temperature, filter_type, blur_radius, gaussian_blur_radius, sharpen_factor, dof_center_x, dof_center_y, dof_blur_amount, dof_falloff_radius):
        if image is not None and filter_type == "Depth of Field":
            width, height = image.size
            center_x = dof_center_x * width
            center_y = dof_center_y * height
            return edit_image(image, red, green, blue, gamma, contrast, brightness, hue, vibrance, temperature, filter_type, blur_radius, gaussian_blur_radius, sharpen_factor, center_x, center_y, dof_blur_amount, dof_falloff_radius)
        else:
            return edit_image(image, red, green, blue, gamma, contrast, brightness, hue, vibrance, temperature, filter_type, blur_radius, gaussian_blur_radius, sharpen_factor, None, None, None, None)

    filter_dropdown.change(update_filter_visibility, inputs=[filter_dropdown], outputs=[blur_radius_slider, gaussian_blur_radius_slider, sharpen_factor_slider, dof_group])

    inputs = [image_input, rgb_red, rgb_green, rgb_blue, gamma_slider, contrast_slider, brightness_slider, hue_slider, vibrance_slider, temperature_slider, filter_dropdown, blur_radius_slider, gaussian_blur_radius_slider, sharpen_factor_slider, dof_center_x_slider, dof_center_y_slider, dof_blur_amount_slider, dof_falloff_radius_slider]
    outputs = [image_output]

    image_input.change(process_image, inputs=inputs, outputs=outputs)
    rgb_red.change(process_image, inputs=inputs, outputs=outputs)
    rgb_green.change(process_image, inputs=inputs, outputs=outputs)
    rgb_blue.change(process_image, inputs=inputs, outputs=outputs)
    gamma_slider.change(process_image, inputs=inputs, outputs=outputs)
    contrast_slider.change(process_image, inputs=inputs, outputs=outputs)
    brightness_slider.change(process_image, inputs=inputs, outputs=outputs)
    hue_slider.change(process_image, inputs=inputs, outputs=outputs)
    vibrance_slider.change(process_image, inputs=inputs, outputs=outputs)
    temperature_slider.change(process_image, inputs=inputs, outputs=outputs)
    filter_dropdown.change(process_image, inputs=inputs, outputs=outputs)
    blur_radius_slider.change(process_image, inputs=inputs, outputs=outputs)
    gaussian_blur_radius_slider.change(process_image, inputs=inputs, outputs=outputs)
    sharpen_factor_slider.change(process_image, inputs=inputs, outputs=outputs)
    dof_center_x_slider.change(process_image, inputs=inputs, outputs=outputs)
    dof_center_y_slider.change(process_image, inputs=inputs, outputs=outputs)
    dof_blur_amount_slider.change(process_image, inputs=inputs, outputs=outputs)
    dof_falloff_radius_slider.change(process_image, inputs=inputs, outputs=outputs)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4fceace2b88c7cf7fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
